# Build a simple LLM Application with LCEL

In this quickstart we'll learn how to build a simple LLM Application with Langchain. This application will translate text from English to another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great wat to get started with Langchain - a lot of features can be built with just some prompting and an LLM call!

Before starting this assignment or project creat an groq api key and load it in .env file where you store all your secret keys.

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv('OPENAI_API_KEY')

groq_api_key =  os.getenv('GROQ_API_KEY')

In [4]:
from langchain_groq import ChatGroq

model = ChatGroq(model = "Gemma2-9b-It",
               groq_api_key = groq_api_key,)

model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000027249432D70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000272494320B0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content = " Translate the following from English to French"),
    HumanMessage(content =" Hello How are you?")
]

result = model.invoke(messages)
result 

AIMessage(content="Bonjour, comment allez-vous ? \n\n\nLet me know if you have any other phrases you'd like translated!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 22, 'total_tokens': 49, 'completion_time': 0.049090909, 'prompt_time': 0.00016089, 'queue_time': 0.01342838, 'total_time': 0.049251799}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b638db76-923d-4d44-bbcf-a7b7c93f282f-0', usage_metadata={'input_tokens': 22, 'output_tokens': 27, 'total_tokens': 49})

In [9]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

"Bonjour, comment allez-vous ? \n\n\nLet me know if you have any other phrases you'd like translated!\n"

In [10]:
## using LCEL we can chain the components. components in the sense model, prompt, output_parser
chain = model|parser 
# ---> first messages are sent to model and then it goes to parser and gothrough all the functionality of output parser and then returns the output 
chain.invoke(messages)

"Bonjour, comment allez-vous ? \n\n\nLet me know if you have any other phrases you'd like translated! 😊 \n"

In [11]:
from langchain_core.prompts import ChatPromptTemplate

genric_template = "Translate the following into {language}:"

prompt =  ChatPromptTemplate.from_messages([
    ('system',genric_template),('user',"{text}")
])



In [15]:
result= prompt.invoke({'language':'French', "text":"Hello How are you?"})
result.to_messages()


[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello How are you?', additional_kwargs={}, response_metadata={})]

In [17]:
chain = prompt|model|parser

chain.invoke({
    'language':'French',
    "text":"Hello How are you?"
})

'Bonjour, comment allez-vous ? \n\n'